In [1]:
import pandas as pd
import time  
from random import uniform  
import re
import io 
import matplotlib.pyplot as plt
from collections import defaultdict
import os

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# fbref Data

In [2]:
# 配置 Selenium WebDriver
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--incognito")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service('chromedriver.exe')

In [3]:
# 映射联赛名称到国家代码
def get_country_code_by_league(league_name):
    league_to_country = {
        "Premier-League": "eng",
        "La-Liga": "es",
        "Ligue-1": "fr",
        "Serie-A": "it",
        "Bundesliga": "de",
    }
    return league_to_country.get(league_name, "unknown")

# 获取球队信息按国家分类
def get_teams_info_by_country(season, league_name):
    # 根据联赛名称获取国家代码
    country = get_country_code_by_league(league_name)
    if country == "unknown":
        print(f"Unsupported league: {league_name}")
        return {}

    # 构造 URL
    driver = webdriver.Chrome(service=service, options=options)
    base_url = f'https://fbref.com/en/comps/Big5/{season}/{season}-Big-5-European-Leagues-Stats'
    driver.get(base_url)

    try:
        # 等待表格加载
        table_xpath = '//*[@id="big5_table"]'
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, table_xpath)))

        # 获取所有行
        row_xpath = '//*[@id="big5_table"]/tbody/tr'
        rows = driver.find_elements(By.XPATH, row_xpath)

        # 提取数据
        teams_info = []
        for row in rows:
            try:
                # 提取链接信息
                link = row.find_element(By.XPATH, './td[1]/a')  # 定位到每一行的链接
                team_id = link.get_attribute('href').split('/')[5]
                team_name = link.get_attribute('href').split('/')[-1].replace('-Stats', '')

                # 提取国家信息
                country_element = row.find_element(By.XPATH, './td[2]/span')
                row_country = country_element.text

                # 仅收集指定国家的数据
                if row_country.lower() == country.lower():
                    teams_info.append({"team_id": team_id, "team_name": team_name, "country": row_country})
            except NoSuchElementException:
                print("No link or country found in this row.")

        # 返回分类结果
        return teams_info

    except TimeoutException:
        print("Table not found or took too long to load.")
        return []
    finally:
        driver.quit()

In [4]:
season = "2023-2024"
categories = ["shooting", "keeper", "passing", "passing_types", "gca", "defense", "possession", "misc"]
leagues = "Premier-League"  #"Premier-League","La-Liga", "Serie-A", "Bundesliga", "Ligue-1"
teams_info = get_teams_info_by_country(season, leagues)
print(len(teams_info)," Teams for ", leagues)
print([team['team_name'] for team in teams_info])

No link or country found in this row.
No link or country found in this row.
No link or country found in this row.
20  Teams for  Premier-League
['Manchester-City', 'Arsenal', 'Liverpool', 'Aston-Villa', 'Tottenham-Hotspur', 'Chelsea', 'Newcastle-United', 'Manchester-United', 'West-Ham-United', 'Crystal-Palace', 'Brighton-and-Hove-Albion', 'Bournemouth', 'Fulham', 'Wolverhampton-Wanderers', 'Everton', 'Brentford', 'Nottingham-Forest', 'Luton-Town', 'Burnley', 'Sheffield-United']


In [5]:
league_codes = {
    "Premier-League": "9",
    "La-Liga": "12",
    "Serie-A": "11",
    "Bundesliga": "20",
    "Ligue-1": "13"
}
base_url_template = "https://fbref.com/en/squads/{team_id}/{season}/matchlogs/c{league_code}/{category}/{team_name}-Match-Logs-{leagues}"
driver = webdriver.Chrome(service=service, options=options)
all_team_category_tables = {}

for team in teams_info:
    team_name = team["team_name"]
    team_category_tables = {}

    for category in categories:
        url = base_url_template.format(
            team_id=team["team_id"],
            season=season,
            leagues=leagues,
            category=category,
            team_name=team_name,
            league_code=league_codes[leagues]
        )
        try:
            time.sleep(uniform(2, 5))  # 模拟随机延迟
            driver.get(url)

            # 等待表格加载
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="matchlogs_for"]'))
            )

            # 解析表格
            table_element = driver.find_element(By.XPATH, '//*[@id="matchlogs_for"]')
            table_html = table_element.get_attribute('outerHTML')
            table = pd.read_html(io.StringIO(table_html))[0]

            # 扁平化列名
            table.columns = ['_'.join(col).strip() for col in table.columns.values]
            team_category_tables[category] = table
            print(f"{team_name} - {category} 表格读取成功")
        except Exception as e:
            print(f"{team_name} - {category} 表格读取失败: {e}")

    all_team_category_tables[team_name] = team_category_tables

# 关闭浏览器
driver.quit()
print("所有数据采集完成！")


Manchester-City - shooting 表格读取成功
Manchester-City - keeper 表格读取成功
Manchester-City - passing 表格读取成功
Manchester-City - passing_types 表格读取成功
Manchester-City - gca 表格读取成功
Manchester-City - defense 表格读取成功
Manchester-City - possession 表格读取成功
Manchester-City - misc 表格读取成功
Arsenal - shooting 表格读取成功
Arsenal - keeper 表格读取成功
Arsenal - passing 表格读取成功
Arsenal - passing_types 表格读取成功
Arsenal - gca 表格读取成功
Arsenal - defense 表格读取成功
Arsenal - possession 表格读取成功
Arsenal - misc 表格读取成功
Liverpool - shooting 表格读取成功
Liverpool - keeper 表格读取成功
Liverpool - passing 表格读取成功
Liverpool - passing_types 表格读取成功
Liverpool - gca 表格读取成功
Liverpool - defense 表格读取成功
Liverpool - possession 表格读取成功
Liverpool - misc 表格读取成功
Aston-Villa - shooting 表格读取成功
Aston-Villa - keeper 表格读取成功
Aston-Villa - passing 表格读取成功
Aston-Villa - passing_types 表格读取成功
Aston-Villa - gca 表格读取成功
Aston-Villa - defense 表格读取成功
Aston-Villa - possession 表格读取成功
Aston-Villa - misc 表格读取成功
Tottenham-Hotspur - shooting 表格读取成功
Tottenham-Hotspur - keeper 表格读取成功
Tottenham-

# Data Preprocessing

In [6]:
new_column_names = ["Date", "Time", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent"]

# 合并所有队伍的数据
merged_tables = []

for team_name, category_tables in all_team_category_tables.items():
    cleaned_tables = []
    for category, table in category_tables.items():
        # 给表格加一个标记列，表示队伍名称
        table['Team'] = team_name
        table.columns = [re.sub(r'^For .+_', '', col) for col in table.columns]   
        cleaned_tables.append(table)
    
    # 合并单个队伍的所有类别表格
    team_merged_table = pd.concat(cleaned_tables, axis=1)
    team_merged_table = team_merged_table[team_merged_table["Aerial Duels_Won%"] != "Aerial Duels"]
    team_merged_table = team_merged_table[team_merged_table["Aerial Duels_Won%"] != "Won%"]
    
    # 去除重复列
    team_merged_table = team_merged_table.loc[:, ~team_merged_table.columns.duplicated()]
    team_merged_table = team_merged_table.dropna(subset='Date')   

    merged_tables.append(team_merged_table)
    
final_merged_table = pd.concat(merged_tables, axis=0)

In [7]:
# Only keep useful data
final_merged_table = final_merged_table.loc[:, ~final_merged_table.columns.str.contains("Match Report")]

# Change Name
column_mapping = {
    "Unnamed: 23_level_0_Ast": "Pass_Ast",
    "Unnamed: 24_level_0_xAG": "Pass_xAG",
    "Unnamed: 25_level_0_xA": "Pass_xA",
    "Unnamed: 26_level_0_KP": "Pass_KP",
    "Unnamed: 27_level_0_1/3": "Pass_1/3",
    "Unnamed: 28_level_0_PPA": "Pass_PPA",
    "Unnamed: 29_level_0_CrsPA": "Pass_CrsPA",
    "Unnamed: 30_level_0_PrgP": "Pass_PrgP",
}
final_merged_table.rename(columns=column_mapping, inplace=True)

defend_column_mapping = {
    "Unnamed: 21_level_0_Int": "Defend_Int",
    "Unnamed: 22_level_0_Tkl+Int": "Defend_Tkl+Int",
    "Unnamed: 23_level_0_Clr": "Defend_Clr",
    "Unnamed: 24_level_0_Err": "Defend_Err",
}
final_merged_table.rename(columns=defend_column_mapping, inplace=True)

## Save Data

In [8]:
# 使用 format 动态生成文件名
os.makedirs(leagues, exist_ok=True)  # Ensure the directory exists
file_path = f"{leagues}/{season}_data.csv"
final_merged_table.to_csv(file_path, index=False, encoding='utf-8-sig')